## P0 Test Suite
#### Original Author: Emil Sekerinski, McMaster University, February 2017
#### Modified by Dom DiPasquale, Patrick Jakubowski, and Mustafa Haddara

In [ ]:
import nbimporter
from Util import compareOutput

## Introduction
The compiler problem is generally solved; that is, the task of translating an input text based on a known grammar and rules describing the desired output has a generally accepted solution. However, due to the vagueness and abstractness of the grammar and rules, the resulting code may be unnecessarily complex or inefficient.

The job of a compiler optimization is to make the resulting code more efficient, either in memory use or cpu time. In project we implement a few optimizations to the `P0` compiler used throughout the course and demonstrate the reductions in memory use or cpu time from the optimized code compared to the code produced by the original compiler.

## Optimizations Implemented
We implemented 4 optimizations. The first two remove unused code to reduce memory overhead, the second two simplify existing code for faster execution. 
1. Removing Unused Procedures
1. Removing Unused Variables 
1. Dead Store Elimination
1. Common Subexpression Elimination

## Ordering
We conduct our individual optimizations serially, in the order that we believed would eliminate the largest chunks of code first: procedures are generally larger than the effective "lifetime" of a variable, which in turn are larger than the sizes of the subexpressions we evaluate in Optimization #4. Thus, when moving down to a "smaller" chunk, we can be sure that this chunk is used by the surrounding code, rather than evaluating variables when the containing procedure is never used. Using this method, we can gaurantee that each sequential optimization has to iterate over the least amount code that we deem useful.

## Examples

### Unused Procedures Elimination

The following example demonstrates unused procedures elimination. An "unused procedure" is when a procedure is defined, but that procedure is never used within the main program's function call graph.

Notice in the example that although we define procedures b, c and e on lines 6, 22 and 42 respectively, we never call these procedures from the main program's function call graph. Notice in the example that procedure b is called within procedure c, but since procedure c is never called from the main program's function call graph, procedure b is also never called from the main program's function call graph. Notice in the example that procedure e is defined in procedure d and procedure d is called from the main program's function call graph, but procedure e is never called from the main program's function call graph.

Future work includes implementing procedure inlining, which would save the overhead of procedure calls. This would eliminate even more defined procedures and as a result, reduce the amount of jal operations.
Future work includes eliminating unused writes to registers, which would save instructions. However, since these writes are typically add or addi operations, the only savings we will see will be from fewer operations; add and addi operations are typically very quick instructions, especially compared to sw operations, which can take far longer.

In [ ]:
compareOutput("""program p;
    var a: integer;
    procedure b(i: integer);
        begin
            a := a + i
        end;
    procedure c(i: integer);
        begin
            a := a + i;
            b(1)
        end;
    procedure d(i: integer);
        procedure e(i: integer);
            begin
                a := a + i
            end;
        begin
            a := a + i
        end;
    begin
        a := a + 1;
        d(1)
    end
    """)

### Dead Store Elimination
The following example demonstrates dead store elimination. A "dead store" is a memory operation that stores a value to a memory location, but that memory location is never read from for the rest of the program. Thus, this store is unused or "dead".

Notice in the example that although we store the value `5` to variable `b` on line 15, we never read that value from that variable. Thus the second `sw` operation to `b` is unneeded.

Future work includes eliminating unused writes to registers, which would save instructions. However, since these writes are typically `add` or `addi` operations, the only savings we will see will be from fewer operations; `add` and `addi` operations are typically very quick instructions, especially compared to `sw` operations, which can take far longer.

In [ ]:
compareOutput("""program p;
    var a,b: integer;
    begin
        b := 4;
        a := 10 + b;
        b := 5;
        write(a)
    end
    """)

Procedures present an interesting complication; since the `$fp` and `$sp` registers are used across procedures, the stores and loads for each of these registers do not happen in the sequence of instructions. We mark these as special case registers, and any `sw` operations writing to these locations are never removed. Additionally, these registers are always used with relative addresses, and those relative addresses are not always consistent throughout the entire use.

For instance, consider the following `P0` code. The `sw` at the beginning of the function block stores the value of `$fp` into the memory location `$sp - 8` (line 6). However, at the end of the function, the same memory location is refered to by the address `$fp - 4` (line 15). If we only considered exact memory locations, we would remove the first `sw` operations at the beginning of the function block, which would lead to incorrect code. 

In [ ]:
compareOutput("""program p;
    procedure f(i: integer);
        begin
        write(i)
        end;
    begin
        f(2)
    end
    """)

### Unused Variable Elimination
The following example demonstrates unused variable elimination; we detect the variables never used and remove them from the program. Since we perform the dead store elimination before this optimization, we are able to detect that variable `b` is never used. Therefore, we are allowed to remove that memory declaration in addition to the declaration for  variable `c`, which is never used (read or written to) in the program.

In the cases of unused variables within function calls, we are able to reduce the amount of stack space allocated by the function (line 12) and must also be careful to remap the used variables to their new memory addresses, ie. if we only allocate 16 bytes on the stack, we should not refer to the value at `$sp - 20`.

In [ ]:
compareOutput("""program p;
    var a,b,c: integer;
    procedure f;
        var m,n,o: integer;
        begin
        o := 1;
        m := 2;
        write(o)
        end;
    begin
        b := 4;
        a := 10;
        b := 5;
        f();
        write(a)
    end
    """)

#### Global Variables and Procedures
Procedures introduce two problems. First and foremost, we must be careful to not remove a global variable which is used exclusively within procedures. Our intial approach consisted of scanning all "blocks", beginning with the main `program` block, looking for uses of variables. However, if a variable is used exclusively within a procedure, we do not detect that it is used, and therefore miss out on this declaration. The solution in this case is to scan the procedures _first_ and then scan the main `program` block.

In the following example, note that although the global variable is never used in the main `program` block, it is used within the procedure `f`, and therefore is not removed. Since all variables are used in this example, the optimizations have no effect on the produced code.

In [ ]:
compareOutput("""program p;
    var a: integer;
    procedure f(i: integer);
        begin
        a := i
        end;
    begin
        f(2)
    end
    """)

The second problem is a result of the `P0` language supporting _shadowing_. Shadowing describes the phenomenon where a local variable has the same name as a global variable, and there hides or "shadows" the global variable. In this case, we must detect that the variable being used is actually a local variable, not a global variable.

The following example demonstrates an unused global variable being removed due to being shadowed.

In [ ]:
compareOutput("""program p;
    var a: integer;
    procedure f(i: integer);
        var a: integer;
        begin
        a := i + 1;
        write(a)
        end;
    begin
        f(2)
    end
    """)

### TODO Pat fill in CSE

The following examples demonstrate all of the optimizations working in conjunction.

In [ ]:
compareOutput("""program p;
      type T = record d,e,f:integer end;
      type A = array[0..9] of integer;
      var c: integer;
      var b: boolean;
      var w: A;
      procedure q(var z: integer);
            type T = record d,e:integer end;
            type A = array[0..9] of integer;
            var y: integer;
            var m,n: boolean;
            var w: T;
            var wd, we: integer;

            begin b:=true; w.d := 9; w.e := 5; w.d := 3 - 1 end;
      begin 
      w[1] := 9; w[2] := 5;
        c := 10;
        c := c + 1;
        write(c);
        q(c)
      end
    """)

In [ ]:
compareOutput("""program p;
  var g: integer;          {global variable}
  procedure q(v: integer); {value parameter}
    var l: integer;        {local variable}
    procedure qx(v: integer); {value parameter}
      var l: integer;        {local variable}
      procedure qxt(v: integer); {value parameter}
        var l: integer;        {local variable}
        begin
          l := 9
        end;
      begin
        l := 9;
        q(l)
      end;
    procedure qc(v: integer); {value parameter}
      var l: integer;        {local variable}
      begin
        l := 9;
        q(l)
      end;
    begin
      l := 9;
      qx(l);
      if l > v then
         write(l)
      else
         write(g)
    end;
  procedure x(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      q(l)
    end;
  begin
    g := 5;
    q(7)
  end
""")

In [ ]:
compareOutput("""program p;
  var g: integer;          {global variable}
  procedure x(v: integer); {value parameter}
    var l: integer;        {local variable}
    procedure x2(v: integer); {value parameter}
    var l: integer;        {local variable}
    begin
      l := 9;
      x2(l)
    end;
    begin
      l := 9;
      x2(l)
    end;
  begin
    g := 5;
    x(g)
  end
""")